In [ ]:
"Incase you don't have these."
!pip install timm
!pip install torchtyping

In [9]:
import torch
import torchvision

import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms

from torchtyping import TensorType, patch_typeguard
from torch.utils.data.dataloader import DataLoader
from typing import Callable, Iterator, Dict
from typeguard import typechecked

from timm.data import create_dataset

patch_typeguard()  # use before @typechecked

In [ ]:
"TODO: Downloading required"
"Get your imagenetmini dataset at:"
"https://drive.google.com/drive/folders/1oudus89CoG9_7r3twbIhw2K_pgVr0D0X?usp=sharing"

"Get your pre-trained resnet50 model (supcon.pth) at:"
"https://www.dropbox.com/s/l4a69ececk4spdt/supcon.pth?dl=0"

In [10]:
image_size = 448
batch_size = 2
projection_dim = 128 #128 for imagenette, 64 for CIFAR10
num_classes = 2

In [11]:
"Code below loads the imagenetmini data set."
tim_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

imagenet_ds_test = create_dataset(name = '', root = 'imagenetmini', transform = tim_transform)

img_testloader = torch.utils.data.DataLoader(
    imagenet_ds_test,
    batch_size=batch_size,
    shuffle=False,
    drop_last=True,
    num_workers=2,
)

In [12]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
model = torchvision.models.resnet50(pretrained=False)
sd = 'supcon.pth'
sd = torch.load(sd, map_location=DEVICE)

"TODO: Implementation required:"
"Load the pre-trained weights into the resnet50 model, supposedly these pre-trained weights were trained"
"using the sought-after supervised contrastive losses."
"Source: https://github.com/HobbitLong/SupContrast"



In [14]:
def inference(loader, model, device):
    feature_vector = []
    labels_vector = []
    
    feature_vector_blocks = [[] for i in range(4)]

    for step, (x, y) in enumerate(loader):
        
        x = x.to(DEVICE)

        # get encoding
        with torch.no_grad():
            "TODO: Implementation required"
            "Use the resnet50-model to extract the features from intermediate layers."
            "Hint: Check layer 1, 2, 3, 4 in "
            "https://pytorch.org/vision/0.8/_modules/torchvision/models/resnet.html"
            
        h = h.detach()
        h_blocks = [h_1, h_2, h_3, h_4]
        for j, i in enumerate(h_blocks):
            i = i.detach()
            feature_vector_blocks[j].extend(i.cpu().detach().numpy())

        feature_vector.extend(h.cpu().detach().numpy())    
        labels_vector.extend(y.numpy())

        if step % 20 == 0:
            print(f"Step [{step}/{len(loader)}]\t Computing features...")
            
        if step == 22:
            break

    feature_vector = np.array(feature_vector)
    labels_vector = np.array(labels_vector)
    
    for i in range(len(feature_vector_blocks)):
        feature_vector_blocks[i] = np.array(feature_vector_blocks[i])
    
    print("Features shape {}".format(feature_vector.shape))
    return feature_vector, labels_vector, feature_vector_blocks

def get_features(context_model, test_loader, device):
    test_X, test_y, test_blocks = inference(test_loader, context_model, device)
    return test_X, test_y, test_blocks 

def create_data_loaders_from_arrays(X_test, y_test, block_test, batch_size):
    test1 = torch.utils.data.TensorDataset(
        torch.from_numpy(block_test[0]), torch.from_numpy(y_test)
    )
    test2 = torch.utils.data.TensorDataset(
        torch.from_numpy(block_test[1]), torch.from_numpy(y_test)
    )
    
    test3 = torch.utils.data.TensorDataset(
        torch.from_numpy(block_test[2]), torch.from_numpy(y_test)
    )
    test4 = torch.utils.data.TensorDataset(
        torch.from_numpy(block_test[3]), torch.from_numpy(y_test)
    )

    test = torch.utils.data.TensorDataset(
        torch.from_numpy(X_test), torch.from_numpy(y_test)
    )
    test_loader = torch.utils.data.DataLoader(
        test, batch_size=batch_size, shuffle=False
    )
    
    return test_loader, test1, test2, test3, test4

In [ ]:
print("### Creating features from pre-trained context model ###")
test_X, test_y, test_blocks = get_features(model, img_testloader, DEVICE)
arr_test_loader, block_1, block_2, block_3, block_4 = create_data_loaders_from_arrays(test_X, test_y, test_blocks, batch_size)

In [ ]:
for step, (images, _) in enumerate(imagenet_ds_test):
    if step != 23:
        continue
    batch = images.cpu().detach().numpy()
    batch = np.moveaxis(batch, 0, -1)
    plt.axis('off')
    plt.imshow(batch)
    plt.imsave('dog.jpg', batch)
    break

In [ ]:
for step, (images, _) in enumerate(block_1):

    if step != 23:
      continue
    print('images.shape:', images.shape)

    image = images.cpu().detach().numpy()
    image = np.moveaxis(image, 0, -1)
    np.save('superblock1.npy', image)

    break

for step, (images, _) in enumerate(block_2):
    if step != 23:
      continue
    print('images.shape:', images.shape)
    image = images.cpu().detach().numpy()

    # put color channels as last dimension 
    image = np.moveaxis(image, 0, -1)
    np.save('superblock2.npy', image)
    
    break

for step, (images, _) in enumerate(block_3):
    if step != 23:
      continue
    print('images.shape:', images.shape)
    image = images.cpu().detach().numpy()

    # put color channels as last dimension 
    image = np.moveaxis(image, 0, -1)
    np.save('superblock3.npy', image)

    break

for step, (images, _) in enumerate(block_4):
    if step != 23:
      continue
    print('images.shape:', images.shape)
    image = images.cpu().detach().numpy()

    # put color channels as last dimension 
    image = np.moveaxis(image, 0, -1)
    np.save('superblock4.npy', image)

    break

Finally, it is valuable for me to know how long it took for you to complete this task:
